In [2]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [3]:
# tf.function
x = tf.Variable(1.0,dtype=tf.float32)

@tf.function(input_signature=[tf.TensorSpec(shape=[],dtype=tf.float32)])
def add_print(a):
    x.assign_add(a)
    tf.print(x)
    return(x)


In [9]:
add_print(tf.constant(3.0))

16


<tf.Tensor: shape=(), dtype=float32, numpy=16.0>

In [47]:
class DemoModule(tf.Module):
    def __init__(self,init_value=tf.constant(0.0),name=None):
        super(DemoModule,self).__init__(name=name)
        self.x = tf.Variable(init_value,dtype=tf.float32,trainable=True)
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[],dtype=tf.float32)])
    def addprint(self,a):
        self.x.assign_add(a)
        tf.print(self.x)
        return(self.x)

In [48]:
demo = DemoModule(init_value=tf.constant(1.0))
result = demo.addprint(tf.constant(5.0))

6


In [51]:
#查看模块中的全部变量和全部可训练变量
print(demo.variables)
print(demo.trainable_variables)

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>,)
(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>,)


In [52]:
print(demo.submodules)

()


In [53]:
#使用tf.saved_model 保存模型，并指定需要跨平台部署的方法
tf.saved_model.save(demo,'F:\study\jupyter\demo',signatures={"serving_default":demo.addprint})

INFO:tensorflow:Assets written to: F:\study\jupyter\demo\assets


In [54]:
#加载模型
demo2 = tf.saved_model.load('F:\study\jupyter\demo')
demo2.addprint(tf.constant(7.0))

13


<tf.Tensor: shape=(), dtype=float32, numpy=13.0>

In [57]:
# 查看模型文件相关信息，红框标出来的输出信息在模型部署和跨平台使用时有可能会用到
!saved_model_cli show --dir F://study//jupyter//demo --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['a'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: serving_default_a:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: 'addprint'
    Option #1
      Callable with:
        Argument #1
          a: TensorSpec(shape=(), dtype=tf.float32, name='

In [58]:
import datetime

# 创建日志
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'F:\study\jupyter\demo\data\demomodule\%s' % stamp
writer = tf.summary.create_file_writer(logdir)

#开启autograph跟踪
tf.summary.trace_on(graph=True, profiler=True) 

#执行autograph
demo = DemoModule(init_value = tf.constant(0.0))
result = demo.addprint(tf.constant(5.0))

#将计算图信息写入日志
with writer.as_default():
    tf.summary.trace_export(
        name="demomodule",
        step=0,
        profiler_outdir=logdir)

Instructions for updating:
use `tf.profiler.experimental.start` instead.
5
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


In [59]:
#启动 tensorboard在jupyter中的魔法命令
%reload_ext tensorboard

In [62]:
from tensorboard import notebook
notebook.list() 
notebook.start("--logdir F://study//jupyter//demo//data//demomodule//")

No known TensorBoard instances running.


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 19280.

In [63]:
import tensorflow as tf
from tensorflow.keras import models,layers,losses,metrics

In [64]:
print(issubclass(tf.keras.Model,tf.Module))
print(issubclass(tf.keras.layers.Layer,tf.Module))
print(issubclass(tf.keras.Model,tf.keras.layers.Layer))

True
True
True


In [65]:
tf.keras.backend.clear_session() 

model = models.Sequential()

model.add(layers.Dense(4,input_shape = (10,)))
model.add(layers.Dense(2))
model.add(layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [66]:
model.variables

[<tf.Variable 'dense/kernel:0' shape=(10, 4) dtype=float32, numpy=
 array([[-0.15386927, -0.19180024,  0.5478456 ,  0.10088301],
        [-0.32741347, -0.19712809, -0.647642  ,  0.30152345],
        [ 0.17416489,  0.11206597, -0.01930743, -0.10894775],
        [-0.52207935, -0.5795536 ,  0.43290794,  0.31012416],
        [ 0.61670744,  0.0299505 , -0.24251038,  0.10362256],
        [-0.09019256,  0.56020856, -0.49521616,  0.58771634],
        [ 0.31877357,  0.15153587,  0.04826951, -0.02834111],
        [ 0.5064782 ,  0.22232401,  0.26235956, -0.00745916],
        [ 0.5071391 , -0.06202966,  0.3899572 ,  0.10422677],
        [ 0.05375278, -0.57990324,  0.25624323,  0.6214781 ]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[-0.1430502 , -0.10323596],
        [ 0.21176887, -0.71684074],
        [-0.8017173 , -0.4186542 ],
        [ 

In [68]:
#冻结第0层的变量,使其不可训练
model.layers[0].trainable = False
model.trainable_variables

[<tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[-0.1430502 , -0.10323596],
        [ 0.21176887, -0.71684074],
        [-0.8017173 , -0.4186542 ],
        [ 0.7542825 , -0.5910821 ]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'dense_2/kernel:0' shape=(2, 1) dtype=float32, numpy=
 array([[ 0.40143895],
        [-0.18917358]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [69]:
model.submodules

(<tensorflow.python.keras.engine.input_layer.InputLayer at 0x212f923d700>,
 <tensorflow.python.keras.layers.core.Dense at 0x212fa096250>)

In [70]:
model.layers

In [71]:
print(model.name)
print(model.name_scope())

sequential
sequential/
